# IMPORTS AND INSTALLS

In [1]:
!pip3 install smogn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler as MMS
from sklearn.preprocessing import StandardScaler as SS
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.decomposition import PCA
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
import smogn

In [3]:
ss=SS()

# DATA CLEANING AND SYNTHETIC DATA PREP
**df -> synthetic dataframe**<br>
**df_org-> orginal dataframe**

In [4]:
df_org=pd.read_csv('../train_data_329x19.csv')
df_org.drop(axis=1,labels='Unnamed: 0',inplace=True)
df_org.dropna(axis=0,inplace=True)
df_org.reset_index(drop=True,inplace=True)
df_org['patient_age']=df_org['patient_age'].astype(int)
df = smogn.smoter(
    
    ## main arguments
    data = df_org,           ## pandas dataframe
    y = 'Fat',          ## string ('header name')
    k =10,                    ## positive integer (k < n)
    samp_method = 'extreme',  ## string ('balance' or 'extreme')

    ## phi relevance arguments
    rel_thres = 0.8,         ## positive real number (0 < R < 1)
    rel_method = 'auto',      ## string ('auto' or 'manual')
    rel_xtrm_type = 'high',   ## string ('low' or 'both' or 'high')
    rel_coef = 2.25          ## positive real number (0 < R)
)
df= df.round(2)
x=df.iloc[:,:13]

y=df.iloc[:,15:16]
y
x_val=df_org.iloc[:,:13]

y_val=df_org.iloc[:,15:16]
df.shape


synth_matrix: 100%|##########| 3/3 [00:01<00:00,  1.54it/s]


(643, 19)

In [5]:
y.describe()

,Fat
count,643.000000
mean,34.202286
std,7.591682
min,2.500000
25%,26.550000
50%,36.000000
75%,41.990000
max,42.050000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 0 to 324
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   patient_age     643 non-null    int64  
 1   patient_gender  643 non-null    int64  
 2   Ht              643 non-null    float64
 3   Wt              643 non-null    float64
 4   HB              643 non-null    float64
 5   UREA            643 non-null    float64
 6   CR              643 non-null    float64
 7   Na              643 non-null    float64
 8   K               643 non-null    float64
 9   FBS             643 non-null    float64
 10  HBA1C           643 non-null    float64
 11  SGOT            643 non-null    float64
 12  SGPT            643 non-null    float64
 13  calories        643 non-null    float64
 14  cho             643 non-null    float64
 15  Fat             643 non-null    float64
 16  pro             643 non-null    float64
 17  fluids          643 non-null    flo

In [7]:
df_org.describe()

,patient_age,patient_gender,Ht,Wt,HB,UREA,CR,Na,K,FBS,HBA1C,SGOT,SGPT,calories,cho,Fat,pro,fluids,NA
count,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.00000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000
mean,55.652308,0.606154,159.876615,66.876862,13.294492,84.096277,1.038708,134.978462,4.088862,83.846154,6.49280,24.186154,26.158954,1659.036923,235.395385,27.226154,66.273846,29.055692,5.673846
std,18.460478,0.489355,6.428141,9.200941,2.177099,36.993946,0.794877,2.754821,0.350768,10.946321,1.91834,10.665496,12.622462,247.451152,112.675699,3.823283,7.452448,136.173058,1.828637
min,10.000000,0.000000,135.000000,25.900000,6.580000,10.000000,0.530000,124.000000,2.400000,83.000000,4.60000,7.500000,6.700000,150.000000,164.000000,2.500000,43.000000,1.000000,0.000000
25%,43.000000,0.000000,156.000000,63.000000,12.300000,41.400000,0.860000,135.000000,4.100000,83.000000,5.29000,22.800000,24.800000,1520.000000,209.000000,25.000000,62.000000,2.500000,5.000000
50%,58.000000,1.000000,160.000000,68.000000,13.700000,106.040000,0.890000,135.000000,4.100000,83.000000,5.80000,22.800000,24.800000,1610.000000,221.000000,26.800000,67.000000,2.900000,6.000000
75%,70.000000,1.000000,163.500000,71.700000,14.800000,106.970000,0.960000,135.000000,4.100000,83.000000,6.97000,22.800000,24.800000,1820.000000,250.000000,30.000000,70.000000,3.000000,7.000000
max,94.000000,1.000000,180.000000,98.500000,17.300000,200.700000,9.780000,157.000000,6.700000,276.000000,14.60000,133.000000,170.000000,2500.000000,2180.000000,42.000000,90.000000,800.000000,8.000000


In [8]:
df.describe()

,patient_age,patient_gender,Ht,Wt,HB,UREA,CR,Na,K,FBS,HBA1C,SGOT,SGPT,calories,cho,Fat,pro,fluids,NA
count,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000,643.000000
mean,33.483670,0.800933,150.623142,52.117947,12.127916,69.252644,0.925599,135.152333,4.176750,83.427683,5.962255,24.245179,24.151851,2058.434308,286.804790,34.202286,69.435148,16.252224,6.824261
std,25.992947,0.399609,10.882352,16.338780,2.128839,40.290939,0.581627,2.441102,0.312203,7.787820,1.492790,7.722998,9.580172,441.911883,95.574506,7.591682,6.430643,97.601789,1.744212
min,9.000000,0.000000,135.000000,25.900000,6.580000,10.000000,0.530000,124.000000,2.400000,83.000000,4.600000,7.500000,6.700000,150.000000,164.000000,2.500000,43.000000,1.000000,0.000000
25%,11.000000,1.000000,138.070000,37.200000,11.250000,28.000000,0.850000,134.940000,4.100000,83.000000,5.080000,22.800000,24.690000,1610.000000,221.000000,26.550000,67.000000,2.500000,6.000000
50%,12.000000,1.000000,147.610000,38.980000,11.900000,105.100000,0.890000,135.000000,4.100000,83.000000,5.660000,22.800000,24.800000,2230.000000,329.740000,36.000000,70.020000,3.000000,8.000000
75%,59.000000,1.000000,160.000000,68.000000,13.715000,106.500000,0.900000,135.065000,4.110000,83.000000,5.970000,22.860000,24.800000,2499.315000,343.005000,41.990000,73.000000,3.500000,8.000000
max,94.000000,1.000000,180.000000,98.500000,17.300000,200.700000,9.780000,157.000000,6.700000,276.000000,14.600000,133.000000,170.000000,2502.790000,2180.000000,42.050000,90.000000,800.000000,8.000000


In [9]:
df.to_csv('oversampled_data.csv')

# TEST TRAIN SPLIT

In [10]:
x_train,x_test,y_train,y_test=train_test_split(x_val,y_val,test_size=0.3);

In [11]:
lr=LinearRegression(fit_intercept=True)
en=ElasticNet()
rf=RandomForestRegressor(n_estimators=500,max_depth=3,criterion='absolute_error')
ab=AdaBoostRegressor(n_estimators=100)
gb=GradientBoostingRegressor(n_estimators=100,max_depth=1)
pca=PCA(whiten=True)
lda=LDA()
gpr=GaussianProcessRegressor(kernel=WhiteKernel()+DotProduct())
lgr=LogisticRegression()
svr=SVR(gamma='auto',kernel='rbf')

In [12]:
lr.fit(x_train,y_train.values.ravel())
en.fit(x_train,y_train.values.ravel())
rf.fit(x_train,y_train.values.ravel())
ab.fit(x_train,y_train.values.ravel())
gb.fit(x_train,y_train.values.ravel())
pca.fit(x_train,y_train.values.ravel())
#lda.fit(x_tr,y_tr.values.reshape(-1,1))
gpr.fit(x_train,y_train.values.ravel())
svr.fit(x_train,y_train.values.ravel())

SVR(gamma='auto')

# TRAIN ACC WITH ORGINAL DATASET

In [13]:
print("TRAIN ACC:")
#fh.write("TRAIN ACC - BEFORE FEATURE SELECTION:\n")
print('LR acc: ',lr.score(x_train,y_train.values.ravel()))
print('ElasticNet acc: ',en.score(x_train,y_train.values.ravel()))
print('RandomForest acc: ',rf.score(x_train,y_train.values.ravel()))
print('AdaBoost acc: ',ab.score(x_train,y_train.values.ravel()))
print('GradientBoost acc: ',gb.score(x_train,y_train.values.ravel()))
#fh.write("Gradient Boosting acc: "+str(gb.score(x_train,y_train.values.ravel()))+"\n")

print("PCA acc: ",pca.score(x_train,y_train.values.ravel()))
print("GPR acc: ",gpr.score(x_train,y_train.values.ravel()))
print("SVR acc: ",svr.score(x_train,y_train.values.ravel()))


TRAIN ACC:
LR acc:  0.5053548628520159
ElasticNet acc:  0.3713945311484067
RandomForest acc:  0.6254926193488889
AdaBoost acc:  0.8070369518326351
GradientBoost acc:  0.6735527568897082
PCA acc:  -56.31183355742265
GPR acc:  0.4997662325814132
SVR acc:  0.3166691898511921


# TEST ACC AFTER TRAIN WITH ORGINAL DATASET

In [14]:
print("TEST ACC:")
#fh.write("TEST ACC - BEFORE FEATURE SELECTION:\n")

print('LR acc: ',lr.score(x_test,y_test.values.ravel()))
print('ElasticNet acc: ',en.score(x_test,y_test.values.ravel()))
print('RandomForest acc: ',rf.score(x_test,y_test.values.ravel()))
print('AdaBoost acc: ',ab.score(x_test,y_test.values.ravel()))
print('GradientBoost acc: ',gb.score(x_test,y_test.values.ravel()))
#fh.write("Gradient Boosting acc: "+str(gb.score(x_test,y_test.values.ravel()))+"\n")
print("PCA acc: ",pca.score(x_test,y_test.values.ravel()))
print("GPR acc: ",gpr.score(x_test,y_test.values.ravel()))
print("SVR acc: ",svr.score(x_test,y_test.values.ravel()))


TEST ACC:
LR acc:  0.4116862680102159
ElasticNet acc:  0.36097884366572763
RandomForest acc:  0.5656718269237957
AdaBoost acc:  0.44166130673153126
GradientBoost acc:  0.5098455366282217
PCA acc:  -64.94978197262783
GPR acc:  0.4266393571090755
SVR acc:  0.05184738089825003


# TRAIN SYNTHETIC TEST ORGINAL

In [15]:
lr=LinearRegression(fit_intercept=True)
en=ElasticNet()
rf=RandomForestRegressor(n_estimators=500,max_depth=3,criterion='absolute_error')
ab=AdaBoostRegressor(n_estimators=100)
gb=GradientBoostingRegressor(n_estimators=100,max_depth=1)
pca=PCA(whiten=True)
lda=LDA()
gpr=GaussianProcessRegressor(kernel=WhiteKernel()+DotProduct())
lgr=LogisticRegression()
svr=SVR(gamma='auto',kernel='rbf')

In [16]:
#x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0);

In [17]:
x

,patient_age,patient_gender,Ht,Wt,HB,UREA,CR,Na,K,FBS,HBA1C,SGOT,SGPT
0,11,1,138.12,37.22,9.24,37.73,0.89,132.96,4.6,83.0,5.30,22.79,24.85
1,11,1,138.09,37.22,9.22,37.80,0.89,132.91,4.6,83.0,5.31,22.85,24.80
2,12,1,138.06,37.21,9.17,38.56,0.89,132.98,4.6,83.0,5.30,22.82,24.90
3,11,1,138.03,37.18,9.18,38.01,0.89,133.00,4.6,83.0,5.29,22.84,24.85
4,12,1,137.78,37.20,9.20,38.70,0.89,133.07,4.6,83.0,5.31,22.76,24.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,57,1,160.00,51.00,13.91,105.15,0.89,135.00,4.1,83.0,5.80,22.80,24.80
321,27,1,164.00,74.00,16.11,105.92,0.86,135.00,4.1,83.0,5.18,22.80,24.80
322,48,1,159.00,68.50,16.57,105.60,0.87,135.00,4.1,83.0,7.31,22.80,24.80
323,85,1,163.00,78.00,16.43,107.76,0.85,135.00,4.1,83.0,5.80,22.80,24.80


In [18]:
y

,Fat
0,40.00
1,39.99
2,40.02
3,39.99
4,39.99
...,...
320,27.00
321,30.00
322,31.00
323,28.00


In [19]:
lr.fit(x,y.values.ravel())
en.fit(x,y.values.ravel())
rf.fit(x,y.values.ravel())
ab.fit(x,y.values.ravel())
gb.fit(x,y.values.ravel())
pca.fit(x,y.values.ravel())
#lda.fit(x_tr,y_tr.values.reshape(-1,1))
gpr.fit(x,y.values.ravel())
svr.fit(x,y.values.ravel())

SVR(gamma='auto')

# ACC WITH SYNTHETIC DATA

In [20]:
print('LR acc: ',lr.score(x,y.values.ravel()))
print('ElasticNet acc: ',en.score(x,y.values.ravel()))
print('RandomForest acc: ',rf.score(x,y.values.ravel()))
print('AdaBoost acc: ',ab.score(x,y.values.ravel()))
print('GradientBoost acc: ',gb.score(x,y.values.ravel()))

#fh.write("Gradient Boosting acc: "+str(gb.score(x,y.values.ravel()))+"\n")
print("PCA acc: ",pca.score(x,y.values.ravel()))
print("GPR acc: ",gpr.score(x,y.values.ravel()))
print("SVR acc: ",svr.score(x,y.values.ravel()))

LR acc:  0.8964747032030403
ElasticNet acc:  0.8481810789282249
RandomForest acc:  0.9471730873569228
AdaBoost acc:  0.9591750766892272
GradientBoost acc:  0.947768790495964
PCA acc:  -55.616529216093554
GPR acc:  0.8961753704312463
SVR acc:  0.9316419545199824


# ACC WITH ACTUAL DATA

In [21]:
print('LR acc: ',lr.score(x_val,y_val.values.ravel()))
print('ElasticNet acc: ',en.score(x_val,y_val.values.ravel()))
print('RandomForest acc: ',rf.score(x_val,y_val.values.ravel()))
print('AdaBoost acc: ',ab.score(x_val,y_val.values.ravel()))
print('GradientBoost acc: ',gb.score(x_val,y_val.values.ravel()))

#fh.write("Gradient Boosting acc: "+str(gb.score(x_val,y_val.values.ravel()))+"\n")
print("PCA acc: ",pca.score(x_val,y_val.values.ravel()))
print("GPR acc: ",gpr.score(x_val,y_val.values.ravel()))
print("SVR acc: ",svr.score(x_val,y_val.values.ravel()))

LR acc:  0.2424668977000256
ElasticNet acc:  -0.10232619106939933
RandomForest acc:  0.5873067605641147
AdaBoost acc:  0.7294439224285478
GradientBoost acc:  0.6075719947853245
PCA acc:  -66.17401946605456
GPR acc:  0.2405159082629974
SVR acc:  0.47932546576924173


# RMSE WHEN PREDICTING WITH ACTUAL DATA

In [22]:

print('LR acc: ',metrics.mean_squared_error(y_val,lr.predict(x_val))**0.5)
print('ElasticNet acc: ',metrics.mean_squared_error(y_val,en.predict(x_val))**0.5)
print('RandomForest acc: ',metrics.mean_squared_error(y_val,rf.predict(x_val))**0.5)
print('AdaBoost acc: ',metrics.mean_squared_error(y_val,ab.predict(x_val))**0.5)
print('GradientBoost acc: ',metrics.mean_squared_error(y_val,gb.predict(x_val))**0.5)

#fh.write("Gradient Boosting acc: "+str(metrics.mean_squared_error(y_val,lr.predict(x_val))**0.5)+"\n")
#print("PCA acc: ",metrics.mean_squared_error(y_val,pca.predict(x_val)**0.5))
print("GPR acc: ",metrics.mean_squared_error(y_val,gpr.predict(x_val)**0.5))
print("SVR acc: ",metrics.mean_squared_error(y_val,svr.predict(x_val)**0.5))

LR acc:  3.3225236245760894
ElasticNet acc:  4.007950352797215
RandomForest acc:  2.4523414984466134
AdaBoost acc:  1.9856189873446346
GradientBoost acc:  2.391372696915038
GPR acc:  496.3621441884985
SVR acc:  497.8446600453895


# CONCLUSION:
- Boosting models perform better
- training with synthetic data and predicting with the actual data provides better results than just training and predicting with the original dataset 